In [1]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')

In [2]:
df = pd.read_csv('IPL Matches 2008-2020.csv')

In [3]:
df.drop(['id','city'],axis=1,inplace=True)
df

,team1,team2,toss_winner,toss_decision,winner
0,Royal Challengers Bangalore,Kolkata Knight Riders,Royal Challengers Bangalore,field,Kolkata Knight Riders
1,Kings XI Punjab,Chennai Super Kings,Chennai Super Kings,bat,Chennai Super Kings
2,Delhi Daredevils,Rajasthan Royals,Rajasthan Royals,bat,Delhi Daredevils
3,Mumbai Indians,Royal Challengers Bangalore,Mumbai Indians,bat,Royal Challengers Bangalore
4,Kolkata Knight Riders,Deccan Chargers,Deccan Chargers,bat,Kolkata Knight Riders
...,...,...,...,...,...
811,Royal Challengers Bangalore,Mumbai Indians,Mumbai Indians,field,Royal Challengers Bangalore
812,Mumbai Indians,Delhi Capitals,Delhi Capitals,field,Mumbai Indians
813,Royal Challengers Bangalore,Sunrisers Hyderabad,Sunrisers Hyderabad,field,Sunrisers Hyderabad
814,Delhi Capitals,Sunrisers Hyderabad,Delhi Capitals,bat,Delhi Capitals


In [4]:
df['team1'] = df['team1'].replace(['Delhi Daredevils'],'Delhi Capitals')
df['team1'] = df['team1'].replace(['Deccan Chargers'],'Sunrisers Hyderabad')
df['team1'] = df['team1'].replace(['Gujarat Lions'],'Chennai Super Kings')
df['team1'] = df['team1'].replace(['Rising Pune Supergiants'],'Rajasthan Royals')
df['team1'] = df['team1'].replace(['Rising Pune Supergiant'],'Rajasthan Royals')

df['team2'] = df['team2'].replace(['Delhi Daredevils'],'Delhi Capitals')
df['team2'] = df['team2'].replace(['Deccan Chargers'],'Sunrisers Hyderabad')
df['team2'] = df['team2'].replace(['Gujarat Lions'],'Chennai Super Kings')
df['team2'] = df['team2'].replace(['Rising Pune Supergiants'],'Rajasthan Royals')
df['team2'] = df['team2'].replace(['Rising Pune Supergiant'],'Rajasthan Royals')

df['toss_winner'] = df['toss_winner'].replace(['Delhi Daredevils'],'Delhi Capitals')
df['toss_winner'] = df['toss_winner'].replace(['Deccan Chargers'],'Sunrisers Hyderabad')
df['toss_winner'] = df['toss_winner'].replace(['Gujarat Lions'],'Chennai Super Kings')
df['toss_winner'] = df['toss_winner'].replace(['Rising Pune Supergiants'],'Rajasthan Royals')
df['toss_winner'] = df['toss_winner'].replace(['Rising Pune Supergiant'],'Rajasthan Royals')

df['winner'] = df['winner'].replace(['Delhi Daredevils'],'Delhi Capitals')
df['winner'] = df['winner'].replace(['Deccan Chargers'],'Sunrisers Hyderabad')
df['winner'] = df['winner'].replace(['Gujarat Lions'],'Chennai Super Kings')
df['winner'] = df['winner'].replace(['Rising Pune Supergiants'],'Rajasthan Royals')
df['winner'] = df['winner'].replace(['Rising Pune Supergiant'],'Rajasthan Royals')

df['winner'].isnull().sum()

4

In [5]:
df['winner'].dropna(axis=0,inplace=True)
df['winner'].unique()

array(['Kolkata Knight Riders', 'Chennai Super Kings', 'Delhi Capitals',
       'Royal Challengers Bangalore', 'Rajasthan Royals',
       'Kings XI Punjab', 'Sunrisers Hyderabad', 'Mumbai Indians',
       'Pune Warriors', 'Kochi Tuskers Kerala', nan], dtype=object)

In [6]:
from sklearn.preprocessing import LabelEncoder
team1 = LabelEncoder()
team2 = LabelEncoder()
toss_winner = LabelEncoder()
toss_decision = LabelEncoder()
winner = LabelEncoder()
df['team1']=team1.fit_transform(df['team1'])
df['team2']=team2.fit_transform(df['team2'])
df['toss_winner']=toss_winner.fit_transform(df['toss_winner'])
df['toss_decision']=toss_decision.fit_transform(df['toss_decision'])
df['winner']=winner.fit_transform(df['winner'])

In [7]:
y=df.drop('winner',axis=1)
X=df['winner']
X

0      4
1      0
2      1
3      8
4      4
      ..
811    8
812    5
813    9
814    1
815    5
Name: winner, Length: 816, dtype: int32

In [8]:
df['winner'].unique()
X=df.drop('winner',axis=1)
y = df['winner']


In [9]:
# from sklearn.model_selection import train_test_split
# X_train,X_test,Y_train,Y_test = train_test_split(X,y,test_size=0.2)

In [10]:
from sklearn import svm
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.naive_bayes import MultinomialNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV


In [11]:
model_params = {
    'svm': {
        'model': svm.SVC(gamma='auto'),
        'params' : {
            'C': [1,10,20],
            'kernel': ['rbf','linear']
        }  
    },
    'random_forest': {
        'model': RandomForestClassifier(),
        'params' : {
            'n_estimators': [1,5,10]
        }
    },
    'logistic_regression' : {
        'model': LogisticRegression(solver='liblinear',multi_class='auto'),
        'params': {
            'C': [1,5,10]
        }
    },
    'naive_bayes_gaussian': {
        'model': GaussianNB(),
        'params': {}
    },
    'naive_bayes_multinomial': {
        'model': MultinomialNB(),
        'params': {}
    },
    'decision_tree': {
        'model': DecisionTreeClassifier(),
        'params': {
            'criterion': ['gini','entropy'],
            
        }
    }     
}

In [12]:
scores = []

for model_name, mp in model_params.items():
    clf =  GridSearchCV(mp['model'], mp['params'], cv=5, return_train_score=False)
    clf.fit(X, y)
    scores.append({
        'model': model_name,
        'best_score': clf.best_score_,
        'best_params': clf.best_params_
    })
    
df1 = pd.DataFrame(scores,columns=['model','best_score','best_params'])
df1

,model,best_score,best_params
0,svm,0.534318,"{'C': 1, 'kernel': 'rbf'}"
1,random_forest,0.510998,{'n_estimators': 5}
2,logistic_regression,0.232837,{'C': 5}
3,naive_bayes_gaussian,0.251234,{}
4,naive_bayes_multinomial,0.183810,{}
5,decision_tree,0.519580,{'criterion': 'entropy'}
